Significant material copied from
https://www.toptal.com/machine-learning/tensorflow-machine-learning-tutorial

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Simple Algerbra

In [12]:
x = tf.placeholder(tf.float32, name="x")
a = tf.constant(5.0, name="a", dtype=tf.float32)
b = tf.constant(-1, name="b", dtype=tf.float32)

y = tf.add(tf.multiply(a, x), b)

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    print(session.run(y, feed_dict={x:[2,2]}))

[9. 9.]


In [3]:
tensor_1d = np.array([0, 1, 2])
tensor = tf.convert_to_tensor(tensor_1d, dtype=tf.float64)
with tf.Session() as session:
    print(session.run(tf.exp(tensor)))
    print(session.run(tf.exp(tensor)[1]))

[1.         2.71828183 7.3890561 ]
2.7182818284590455


## Matrix Ops

In [4]:
x = tf.convert_to_tensor(
    np.array(
        [
            (1, 2, 3),
            (4, 9, 6),
            (7, 8, 5)
        ]), tf.float32)

m_tranpose = tf.transpose(x)
m_mul = tf.matmul(x, x)
m_det = tf.matrix_determinant(x)
m_inv = tf.matrix_inverse(x)
m_solve = tf.matrix_solve(x, [[1], [1], [1]])

with tf.Session() as session:
    print (session.run(m_tranpose))
    print (session.run(m_mul))
    print (session.run(m_inv))
    print (session.run(m_det))
    print (session.run(m_solve))

[[1. 4. 7.]
 [2. 9. 8.]
 [3. 6. 5.]]
[[ 30.  44.  30.]
 [ 82. 137.  96.]
 [ 74. 126.  94.]]
[[ 0.05769233 -0.26923078  0.28846157]
 [-0.423077    0.30769232 -0.11538462]
 [ 0.5961539  -0.11538462 -0.01923078]]
-51.999985
[[ 0.0769231 ]
 [-0.23076926]
 [ 0.46153852]]


In [5]:
red_sum_0 = tf.reduce_sum(x)
red_sum = tf.reduce_sum(x, axis=1)

red_prod_0 = tf.reduce_prod(x)
red_prod = tf.reduce_prod(x, axis=1)

red_min_0 = tf.reduce_min(x)
red_min = tf.reduce_min(x, axis=1)

red_max_0 = tf.reduce_max(x)
red_max = tf.reduce_max(x, axis=1)

red_mean_0 = tf.reduce_mean(x)
red_mean = tf.reduce_mean(x, axis=1)

with tf.Session() as session:
    print ("Reduce sum without passed axis parameter: ", session.run(red_sum_0))
    print ("Reduce sum with passed axis=1: ", session.run(red_sum))

    print ("Reduce product without passed axis parameter: ", session.run(red_prod_0))
    print ("Reduce product with passed axis=1: ", session.run(red_prod))

    print ("Reduce min without passed axis parameter: ", session.run(red_min_0))
    print ("Reduce min with passed axis=1: ", session.run(red_min))

    print ("Reduce max without passed axis parameter: ", session.run(red_max_0))
    print ("Reduce max with passed axis=1: ", session.run(red_max))

    print ("Reduce mean without passed axis parameter: ", session.run(red_mean_0))
    print ("Reduce mean with passed axis=1: ", session.run(red_mean))

Reduce sum without passed axis parameter:  45.0
Reduce sum with passed axis=1:  [ 6. 19. 20.]
Reduce product without passed axis parameter:  362880.0
Reduce product with passed axis=1:  [  6. 216. 280.]
Reduce min without passed axis parameter:  1.0
Reduce min with passed axis=1:  [1. 4. 5.]
Reduce max without passed axis parameter:  9.0
Reduce max with passed axis=1:  [3. 9. 8.]
Reduce mean without passed axis parameter:  5.0
Reduce mean with passed axis=1:  [2.        6.3333335 6.6666665]


## Digging In

In [6]:
x = tf.constant(-2.0, name="x", dtype=tf.float32)
a = tf.constant(5.0, name="a", dtype=tf.float32)
b = tf.constant(13.0, name="b", dtype=tf.float32)


y = tf.Variable(tf.add(tf.multiply(a, x), b))


init = tf.global_variables_initializer()


with tf.Session() as session:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs", session.graph)


    session.run(init)
    print(session.run(y))

3.0


## KNN

In [7]:
ndim = 20
ntrain = 1000
ntest = 100
assert ndim>=2

# Generate some data to work with
X_train = np.random.rand(ntrain,ndim)
y_train = np.where(X_train[:,0]>X_train[:,1], 1, 0).reshape(ntrain,1)
X_test = np.random.rand(ntest,ndim)
y_test = np.where(X_test[:,0]>X_test[:,1], 1, 0).reshape(ntest,1)

In [8]:
# Single Neighbor
train_pl = tf.placeholder("float", [None, ndim])
test_pl = tf.placeholder("float", [ndim])

deltas = train_pl-test_pl
distance = tf.reduce_sum(tf.square(deltas), axis=1)
nearest = tf.argmin(distance, 0)

with tf.Session() as tf_session:
    missed = 0

    for i in range(len(y_test)):
        knn_index = tf_session.run(nearest, feed_dict={train_pl: X_train, test_pl: X_test[i]})
#         print("Predicted class {} -- True class {}".format(y_train[knn_index], y_test[i]))
        if y_train[knn_index] != y_test[i]:
            missed += 1

#     tf.summary.FileWriter("logs", tf_session.graph)
    print("Missed: {} -- Total: {}".format(missed, len(y_test)))

Missed: 30 -- Total: 100


In [9]:
# Weighted by Distance (sort a gaussian mixture model)
train_pl = tf.placeholder("float", [None, ndim])
train_label_pl = tf.placeholder("float", [None, 1])
test_pl = tf.placeholder("float", [ndim])

deltas = train_pl-test_pl
distance = tf.reduce_sum(tf.square(deltas), axis=1)
weight = (1/distance)/tf.reduce_sum(1/distance)
proba = tf.reduce_sum(weight*tf.transpose(train_label_pl))
pred = tf.round(proba)

feed_dict={train_pl: X_train, test_pl: X_test[i], train_label_pl: y_train}

with tf.Session() as tf_session:
    missed = 0

    for i in range(len(y_test)):
        feed_dict={train_pl: X_train, test_pl: X_test[i], train_label_pl: y_train}
#         print(X_test[i])
#         print("deltas:\t", tf_session.run(deltas, feed_dict))
#         print("distance:\t", tf_session.run(distance, feed_dict))
#         print("weight:\t", tf_session.run(weight, feed_dict))
#         print("proba:\t", tf_session.run(proba, feed_dict))
        pred_value = tf_session.run(pred, feed_dict)
#         print(int(pred_value))
#         print(int(y_test[i]))
        if int(pred_value) != int(y_test[i]):
            missed += 1
    print("Missed: {} -- Total: {}".format(missed, len(y_test)))

Missed: 23 -- Total: 100


## Linear Regression

In [10]:
train_dataset, train_values = generate_test_values()

NameError: name 'generate_test_values' is not defined

In [ ]:
x = tf.placeholder(tf.float32, [None, 3], name="x")
W = tf.Variable(tf.zeros([3, 1]), name="W")
b = tf.Variable(tf.zeros([1]), name="b")
y = tf.placeholder(tf.float32, [None, 1])

model = tf.add(tf.matmul(x, W), b)
cost = tf.reduce_mean(tf.square(y - model))
train = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    feed_dict={
            x: train_dataset,
            y: train_values
    }

    for _ in range(iterations):
        session.run(train, feed_dict)

    print("cost = {}".format(session.run(cost, feed_dict)))
    print("W = {}".format(session.run(W)))
    print("b = {}".format(session.run(b)))